# Flight Path Plot

Plots an interactive chart of the flight path for an aircraft given its address, plotting East-West on the X axis, North-South on the Y axis and altitude on the Z-axis. The origin on the ground at X = 0, Y = 0 is set to the latitude and longitude for the first point in the aircraft flight path.

The report draws on data from the aircraft tracking database so capture of aircraft flight path data must be enabled using the following tracking application settings:

| Setting         | Value |
| --------------- | ----- |
| EnableSqlWriter | true  |
| TrackPosition   | true  |

To use the plotter:

- Capture some tracking data using the tracking application
- Once the tracking session is complete, use the SQL query "summarise-flight-paths.sql" to provide a list of aircraft with tracking information and select one from the list
- Set the values in the next cell to define the aircraft, time slice and mapbox token (if the ground is to be textured)

In [ ]:
from datetime import datetime

# The 24-bit ICAO address of the aircraft
aircraft_address = ""

# Optionally, the start and end of the time slice to be charted
# Use this syntax to set a value, replacing the date and time with the one of interest: datetime(2025, 10, 15, 19, 40, 15)
start_timestamp = None
end_timestamp = None

# To texture the ground using a street map, set the following to a valid Mapbox API key. If left blank, the ground will not
# be textured
token = ""

In [ ]:

EARTH_RADIUS = 6371000.0

MAX_SEGMENT_GAP_SECONDS = 90

TITLE = dict(
    x=0.5,                  # horizontal centre (0 = left, 0.5 = middle, 1 = right)
    xanchor='center',       # anchor the title to the centre position
    yanchor='top',          # keep it above the chart
    font=dict(size=18)
)

COLOURBAR = dict(
    title="Altitude (m)",
    x=1.05,
    xanchor="left",
    y=0.5
)

LIGHTING = dict(
    ambient=0.6,     # general scene light
    diffuse=0.8,     # brightness on lit surfaces
    specular=0.3,    # shininess
    roughness=0.5,   # 0 = mirror smooth, 1 = matte
    fresnel=0.2      # reflectivity edge highlight
)

LIGHTPOSITION = dict(
    x=2000,          # east-west offset in local coords
    y=0,             # north-south offset
    z=8000           # altitude of the light source (sun height)
)

LEGEND = dict(
    x=0.02,
    y=0.98, 
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="rgba(0,0,0,0.2)",
    borderwidth=1
)

In [ ]:
%run pathutils.ipynb
%run database.ipynb

In [ ]:
import pandas as pd

def load_flight_path(address, from_timestamp=None, to_timestamp=None):
    """
    Load a flight path by aircraft address, optional taking only the specified time slice

    :param address: 24-bit aircraft ICAO address
    :param from_timestamp: Date and time for the start of the time slice
    :param to_timestamp: Date and time for the end of the time slice
    :return: A DataFrame containing the flight path
    """
    # Construct and run the query for the specified address
    query = construct_query("tracker", "reports", "list-flight-path.sql", { "ADDRESS": address })
    df = query_data("tracker", query)

    # Convert the timestamp to datetime and filter based on the timestamp limits
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce", utc=False)
    df = df.dropna(subset=["Latitude","Longitude","Altitude","Distance","Timestamp"]).reset_index(drop=True)

    if from_timestamp:
        df = df[(df["Timestamp"] > from_timestamp)]

    if to_timestamp:
        df = df[(df["timestamp"] < to_timestamp)]

    # Convert altitude to metres and make sure all properties are numeric
    df["Altitude"] = pd.to_numeric(df["Altitude"], errors="coerce") * 0.3048
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
    df["Distance"] = pd.to_numeric(df["Distance"], errors="coerce")

    return df

In [ ]:
import numpy as np

def calculate_z_range(df):
    """
    Calculate the minimum and maximum Z (altitude) range for

    :param df: DataFrame containing the flight path 
    :return: A tuple of the minimum and maximum Z-values for scaling the chart
    """

    # Calculate the range
    altitude = df["Altitude"].to_numpy()
    zmin = float(np.nanmin(altitude))
    zmax = float(np.nanmax(altitude))

    # Pad it (10% or a minimum of 250m padding)
    zpad = max((zmax - zmin) * 0.1, 250.0)
    zmin = max(0.0, zmin - zpad)
    zmax = zmax + zpad

    return zmin, zmax

In [ ]:
def segment_on_gaps(df):
    """
    Identify gaps in flight path data that exceed the configured threshold and break the data into multiple
    segments at these gaps

    :param df: DataFrame containing the flight path 
    :return: A list of 
    """

    # Make sure the data's sorted by timestamp
    s = df.sort_values("Timestamp").reset_index(drop=True).copy()

    # Calculate the difference between each point
    dt = s["Timestamp"].diff().dt.total_seconds().fillna(0)

    # Generate a boolean series indicating if there's a gap and use cumsum to generate a group identifier
    # that increases by 1 every time there's a gap
    group_id = (dt > MAX_SEGMENT_GAP_SECONDS).cumsum()

    # Split the DataFrame by groups
    return [seg.reset_index(drop=True) for _, seg in s.groupby(group_id)]

In [ ]:
import numpy as np

def coordinates_to_local_xy(latitude, longitude):
    """
    Equirectangular (Plate Carrée) Projection

    Convert latitude and longitude into X (East-West) and Y (North-South) coordinates. X increases
    Eastwards and Y increases Northwards relative to a reference point taken as the first point in
    the trace

    :param latitude: A Series of latitude values 
    :param longitude: A Series of latitude values 
    :return: A tuple of two NumPy arrays for the X and Y coordinates
    """
    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    # Convert the points to radians
    latitude_rad = np.radians(latitude)
    longitude_rad = np.radians(longitude)
    ref_latitude_rad = np.radians(latitude[0])
    ref_longitude_rad = np.radians(longitude[0])

    # Calculate the differences between the latitude and the reference point
    delta_longitude = longitude_rad - ref_longitude_rad
    delta_latitude = latitude_rad - ref_latitude_rad

    # Calculate to local distance in meters: x = East-West, y = North-South
    x = delta_longitude * np.cos(ref_latitude_rad) * EARTH_RADIUS   
    y = delta_latitude * EARTH_RADIUS

    return x, y

In [ ]:
import numpy as np

def flight_path_bounding_box(latitude, longitude, pad_ratio=0.05):
    """
    Calculate the the smallest rectangle (in latitude/longitude degrees) on the ground plane that fully
    contains all points in the flight path, with an optional “breathing space” margin around it

    :param latitude: A Series of latitude values 
    :param longitude: A Series of latitude values 
    :return: A tuple of two NumPy arrays for the X and Y coordinates
    """

    # Calculate the latitude extents
    latitude = np.asarray(latitude, float)
    minimum_latitude, maximum_latitude = latitude.min(), latitude.max()

    # Calculate the longitude extents
    longitude = np.asarray(longitude, float)
    minimum_longitude, maximum_longitude = longitude.min(), longitude.max()

    # Pad a little so the path isn’t tight to the edges
    latitude_difference = (maximum_latitude - minimum_latitude) or 1e-6
    longitude_difference = (maximum_longitude - minimum_longitude) or 1e-6
    latitude_padding = latitude_difference * pad_ratio
    longitude_padding = longitude_difference * pad_ratio

    # Calculate the co-ordinates of the bounding box
    north = maximum_latitude + latitude_padding
    south = minimum_latitude - latitude_padding
    east = maximum_longitude + longitude_padding
    west = minimum_longitude - longitude_padding

    return (north, south, east, west)

In [ ]:
import requests

def fetch_mapbox_static(bbox, width=1280, height=1280, style="streets-v12", token="", scale=2):
    """
    Retrieve a map image covering the flight path bounding box

    :param bbox: A tuple of north, south, east, west coordinates for the bounding box, expressed in degrees latitude/longitude
    :param width: Image width in pixels
    :param height: Image height in pixels
    :param style: Mapbox map style
    :param token: API token
    :param scale: Controls DPI of downloaded image
    :return: A tuple of two NumPy arrays for the X and Y coordinates
    """

    # Unpack the bounding box
    north, south, east, west = bbox

    # Construct the part of the URL that controls image size and resolution
    size_seg = f"{width}x{height}" + ("@2x" if scale == 2 else "")
    
    # Construct the final URL
    url = (f"https://api.mapbox.com/styles/v1/mapbox/{style}/static/"
           f"[{west},{south},{east},{north}]/{size_seg}?access_token={token}")
    
    # Request the map image and return its content
    r = requests.get(url, timeout=30); r.raise_for_status()
    return r.content

In [ ]:
import numpy as np

def curtain_mesh(x, y, z):
    """
    Build a vertical 'curtain' mesh under a polyline (x,y,z). For every point in the flight, the top vertex
    is the position of the aircraft and the bottom vertex is the same (x, y) but on the ground. Between
    consecutive segments, the vertices are joined by triangles

    :param x: Series of X coordinates
    :param y: Series of Y coordinates
    :param z: Series of altitudes
    :return: A tuple of:
             X, Y, Z: Vertex coordinates
             I, J, K: Indices into the vertex coordinate arrays defining triangles that Plotly can plot
    """
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.asarray(z)

    # Allocate space for top and bottom vertices - note this is twice the length of the flight path
    # because each point will generate two points, one at the aircraft altitude and one on the ground
    # directly below it
    X = np.empty(2 * len(x))
    Y = np.empty(2 * len(y))
    Z = np.empty(2 * len(z))

    # For each point, make a top vertex (even index) and a bottom vertex (odd index). The following results
    # in the following NumPy array:
    #
    # X = [x0, x0, x1, x1, x2, x2 ...]
    # Y = [y0, y0, y1, y1, y2, y2 ...]
    # Y = [z0, 0.0, z1, 0.0, z2, 0.0 ...]
    #
    X[0::2], X[1::2] = x, x
    Y[0::2], Y[1::2] = y, y
    Z[0::2], Z[1::2] = z, 0.0

    i=j=k=[]
    i=[]
    j=[]
    k=[]

    # Build a set of indices into X, Y, Z that define a set of triangles that Plotly can plot to produce
    # the curtain mesh
    for t in range(len(x)-1):
        # Indices of the four vertices forming one quad
        t0 = 2*t            # top of point t
        b0 = 2*t + 1        # bottom of point t
        t1 = 2*(t+1)        # top of point t+1
        b1 = 2*(t+1)+1      # bottom of point t+1

        # Split the quad into two triangles: (t0, b0, b1) and (t0, b1, t1)
        i += [t0, t0]
        j += [b0, b1]
        k += [b1, t1]

    return X, Y, Z, np.array(i), np.array(j), np.array(k)

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_flight_ribbon(df, x, y, zmin, zmax):
    """
    Plot a segment of the flight path

    :param df: DataFrame containing the x, y coordinates and altitude for each point in the flight path
    :param zmin: Minimum z (altitude) value for scaling the z-axis
    :param zmax: Minimum z (altitude) value for scaling the z-axis
    :return: A plotly figure for this segment
    """

    # Calculate the X, Y, Z coordinates of the vertices in the curtain mesh and the indices into those vertex
    # arrays that define triangles for plotting
    X, Y, Z, I, J, K = curtain_mesh(x, y, df["Altitude"])

    # Create a new figure
    fig = go.Figure()

    # Add the ribbon to the figure
    fig.add_trace(
        go.Mesh3d(
            x=X, y=Y, z=Z,
            i=I, j=J, k=K,
            opacity=0.85,
            flatshading=True,
            name="Ribbon",
            intensity=Z,
            colorscale="Plasma",
            colorbar=COLOURBAR,
            lighting=LIGHTING,
            lightposition=LIGHTPOSITION,
            showscale=True
        )
    )

    # Add the flight path and ground trace
    fig.add_trace(go.Scatter3d(x=x, y=y, z=df["Altitude"], mode="lines", line=dict(width=4), name="Flight path"))
    fig.add_trace(go.Scatter3d(x=x, y=y, z=np.zeros_like(df["Altitude"]), mode="lines", line=dict(width=2, dash="dash"), name="Ground trace"))

    # Apply z-range explicitly and turn off autorange
    fig.update_scenes(aspectmode="data", zaxis_autorange=False, zaxis_range=[zmin, zmax])

    return fig

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import math

def attach_info_strip(fig_3d, info, columns=3, row_heights=(0.82, 0.18), height=820):
    """
    Takes an existing 3D figure (with a single scene) and returns a new figure
    that has the 3D scene on row 1 and a Table 'info strip' on row 2.

    :param fig_3d: The 3D figure containing the flight path
    :param info: Dictionary of flight properties e.g. {"Flight": "BA123", "Airline": "British Airways", ...}
    :param columns: Number of information items per line
    :param row_heights: A tuple of the fractional row heights for the 3D plot and information strip
    :param height: Height of the combined figure, in pixels
    """
    # Create a 2-row figure: top = 3D scene, bottom = domain for Table
    fig = make_subplots(
        rows=2, cols=1,
        specs=[[{"type": "scene"}],
               [{"type": "domain"}]],
        row_heights=list(row_heights),
        vertical_spacing=0.06
    )

    # Move all traces from the original fig into row 1
    for tr in fig_3d.data:
        fig.add_trace(tr, row=1, col=1)

    # Copy layout without overriding the subplot's scene domain
    if getattr(fig_3d.layout, "scene", None) is not None:
        scene_json = fig_3d.layout.scene.to_plotly_json()
        scene_json.pop("domain", None)
        fig.update_layout(scene=scene_json)

    # Keep the 3D plot title/legend/coloraxis if present
    if getattr(fig_3d.layout, "title", None) is not None:
        fig.update_layout(title=fig_3d.layout.title)
    if getattr(fig_3d.layout, "legend", None) is not None:
        fig.update_layout(legend=fig_3d.layout.legend)
    if getattr(fig_3d.layout, "coloraxis", None) is not None:
        fig.update_layout(coloraxis=fig_3d.layout.coloraxis)

    # Ensure the figure is tall enough and margins sufficient
    fig.update_layout(
        height=height,
        margin=dict(l=60, r=60, t=70, b=20)
    )

    # Build a compact table from the info dictionary
    pairs = [f"<b>{k}</b>  {v}" for k, v in info.items()]

    rows_needed = math.ceil(len(pairs) / columns)
    table_rows = []
    for r in range(rows_needed):
        row_items = pairs[r*columns:(r+1)*columns]
        # pad to full width for a tidy grid
        row_items += [""] * (columns - len(row_items))
        table_rows.append(row_items)

    # Transpose for go.Table (values is list-of-columns)
    cols = list(map(list, zip(*table_rows)))

    fig.add_trace(
        go.Table(
            header=dict(
                values=[""] * columns,
                line=dict(width=0),
                fill_color="rgba(0,0,0,0)",
                height=8
            ),
            cells=dict(
                values=cols,
                align="left",
                height=22,
                line=dict(color="rgba(0,0,0,0.08)", width=1),
                fill_color=[["rgba(255,255,255,0.0)"] * rows_needed]*columns
            )
        ),
        row=2, col=1
    )

    # A faint border box feel for the strip
    fig.update_layout(
        annotations=list(fig.layout.annotations) + [
            dict(
                xref="paper", yref="paper",
                x=0.0, y=0.0, xanchor="left", yanchor="bottom",
                text="", showarrow=False,
                bgcolor="rgba(255,255,255,0.65)",
                bordercolor="rgba(0,0,0,0.15)", borderwidth=1,
                xshift=10, yshift=100
            )
        ]
    )

    return fig

In [ ]:
def calculate_floor_triangles(H, W):
    """
    Calculate a set of triangles covering the floor of the chart

    :param H: Height of the floor
    :param W: Width of the floor
    :return: Tuple of triangle vertex coordinates
    """

    # Initialise the vertex arrays for each triangle
    I = []
    J = []
    K = []

    # This flattens 2D grid coordinates into a 1D index
    idx = lambda r, c: r * W + c

    # Loop over each cell in the grid
    for r in range(H-1):
        for c in range(W-1):
            # Each cell is defined by its top-left co-ordinate (r, c) and is rectangular, so two triangles are needed
            # to fill it. I, J, K are the vertices defining each triangle
            I += [idx(r,c), idx(r,c)]
            J += [idx(r+1,c), idx(r+1,c+1)]
            K += [idx(r+1,c+1), idx(r, c+1)]

    # Return the vertex lists as NumPy arrays
    return np.array(I), np.array(J), np.array(K)

In [ ]:
import numpy as np
from PIL import Image
import io
import plotly.graph_objects as go

def add_textured_ground_bytes(fig, img_bytes, x_min, x_max, y_min, y_max, z_floor, max_px=160):
    """
    Paste the static map retrieved from Mapbox onto the floor of the 3D scene at the specified Z coordinate

    :param fig: The 3D figure to paste into
    :param img_bytes: The bytes for the image
    :param x_min: Minimum X for the floor
    :param x_max: Maximum X for the floor
    :param y_min: Minimum Y for the floor
    :param y_min: Minimum Y for the floor
    :param z_floor: The Z coordinate for the floor
    :param max_px: Downsampling limit for the longest side of the map image
    """
    # Convert the images bytes to a PIL image
    im = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    w, h = im.size

    # Calculate a scaling factor so the image can be scaled to keep the number of triangles rendered to a
    # more reasonable number
    scale = max(w, h) / float(max_px) if max(w, h) > max_px else 1.0
    if scale > 1:
        # Scale the image using Image.LANCZOS, one of Pillow’s high-quality resampling filters, to calculate the
        # output pixels
        im = im.resize((int(round(w/scale)), int(round(h/scale))), Image.LANCZOS)

    # Convert the image into a NumPy array representing the pixel data and get the height and width
    im = np.asarray(im)
    H, W, _ = im.shape

    # Create W x evenly spaced X positions and H x evenly spaced Y positions on the floor
    xs = np.linspace(x_min, x_max, W)   # [x0, x1, x2, ...]
    ys = np.linspace(y_max, y_min, H)   # [y0, y1, y2, ...]

    # Create a 2D coordinate grid so we can map a value at each x,y location
    XX, YY = np.meshgrid(xs, ys)

    # Create a constant plane at the floor - this is a matching 2D array of Z coordinates
    ZZ = np.full_like(XX, z_floor, dtype=float)

    # Calculate the triangular surface for the floor
    I, J, K = calculate_floor_triangles(H, W)

    # ravel() unrolls the 2D coordinate grid into the 1D array that Plotly will understand
    Xv, Yv, Zv = XX.ravel(), YY.ravel(), ZZ.ravel()

    # Likewise, each point in the image is (H, W, [R, G, B]) - flatten this to a list of RGB values
    rgb = im.reshape(-1, 3)
    vertexcolor = [f"rgb({r},{g},{b})" for r, g, b in rgb]

    # And finally draw the map on the floor!
    fig.add_trace(go.Mesh3d(
        x=Xv, y=Yv, z=Zv,
        i=I, j=J, k=K,
        name="Map",
        vertexcolor=vertexcolor,
        flatshading=True,
        showscale=False,
        lighting=dict(ambient=0.9, diffuse=0.2),
        hoverinfo="skip",
        opacity=1.0
    ))

In [ ]:
def save_html(fig, aircraft_address, start_time):
    """
    Export the plot to an interactive HTML file

    :param fig: Figure to export
    :param aircraft_address: Used in naming the output file
    :param start_time: Start time for the flight path
    """
    export_path = get_export_folder_path()
    timestamp = start_time.strftime("%Y-%m-%d-%H-%M-%S")
    path = export_path / f"{timestamp}-{aircraft_address}.html"
    fig.write_html(path, include_plotlyjs="cdn")

In [ ]:
# Load the flight path data and calculate east-west (x) and north-south (y) coordinates and the altitude limits
df = load_flight_path(aircraft_address, start_timestamp, end_timestamp)
x, y = coordinates_to_local_xy(df["Latitude"], df["Longitude"])
zmin, zmax = calculate_z_range(df)

# Get the data for the info panel
information = {
    "Callsign": df["Callsign"].iloc[0],
    "Registration": df["Registration"].iloc[0],
    "Model":  df["Model"].iloc[0],
    "Flight": df["FlightIATA"].iloc[0],
    "Airline": df["AirlineName"].iloc[0],
    "Route": df["Route"].iloc[0],
    "Start": df["Timestamp"].min().strftime("%Y-%m-%d %H:%M:%S"),
    "End": df["Timestamp"].max().strftime("%Y-%m-%d %H:%M:%S")
}

# add_info_panel(fig, **info_kwargs)
callsign = df["Callsign"].iloc[0]
flight_number = df["FlightIATA"].iloc[0]

# Construct the title
title = f"Flight Path for Aircraft {aircraft_address}"

# Segment to avoid long straight connectors across time gaps
segments = segment_on_gaps(df)

# Create a plot and set the title and legend properties
fig = go.Figure()
fig.update_layout(
    title=dict(
        text=title,
        x=0.5,                  # horizontal centre
        xanchor='center',       # anchor the title to the centre position
        yanchor='top',          # keep it above the chart
        font=dict(size=18)      # optional: make it a bit larger
    ),
    legend=LEGEND
)

# Build a single figure and add each segment as its own ribbon
for idx, seg in enumerate(segments, 1):
    if len(seg) > 2: 
        title = f"{seg["Callsign"].iloc[0] or aircraft_address} — segment {idx}"
        f = plot_flight_ribbon(seg.rename(columns={"Timestamp": "time"}), x, y, zmin, zmax)

        # Merge traces from f into fig
        for tr in f.data:
            fig.add_trace(tr)

# If the token is specified, use MapBox to texture the ground
if token:
    # Compute the map bounding box from the track data
    bbox = flight_path_bounding_box(df["Latitude"].to_numpy(), df["Longitude"].to_numpy(), pad_ratio=0.06)

    # Fetch a static image for the map
    png_bytes = fetch_mapbox_static(bbox, width=1024, height=1024, style="streets-v12", token=token)
                                    
    # add as textured floor
    z_floor = max(0.0, float(zmin))
    add_textured_ground_bytes(fig, png_bytes, x.min(), x.max(), y.min(), y.max(), z_floor, max_px=512)

# Set Z-scaling and show the plot
fig.update_scenes(zaxis_autorange=False, zaxis_range=[float(zmin), float(zmax)])

# Attach the info strip
final_fig = attach_info_strip(fig, information, columns=3)

# Save to interactive HTML
save_html(final_fig, aircraft_address, df["Timestamp"].iloc[0])

# Show the figure
final_fig.show()